# Downloading biological data

Below is the adaptation of the part of the [MultilayerNexus](https://github.com/cirillodavide/MultilayerNexus.git) project for downloading various biological datasets.

In [ ]:
#| default_exp download_data

## 0. Import required packages

In [ ]:
import requests
import zipfile
from datetime import datetime
from tqdm import tqdm
import json
from pathlib import Path
import os
from itertools import combinations
import re
import pandas as pd
import ast
import rdflib
import multiprocessing
from joblib import Parallel, delayed
import gzip
import shutil
import cobra
import urllib.request
from collections import defaultdict

## 1. BioGRIG interactions

In [ ]:
protein_interaction_techniques = {
    "Affinity Capture-Luminescence",
    "Affinity Capture-MS",
    "Affinity Capture-Western",
    "Biochemical Activity",
    "Co-crystal Structure",
    "Co-fractionation",
    "Co-localization",
    "Co-purification",
    "Far Western",
    "FRET",
    "PCA",
    "Protein-peptide",
    "Reconstituted Complex",
    "Two-hybrid",
}

def ensure_dir(file_path):
   file_path.parent.mkdir(parents=True, exist_ok=True)

def download_file(url, path, file_size):
    ensure_dir(path)
    chunk_size = 1024  # 1KB
    with requests.get(url, stream=True) as r, open(path, 'wb') as f, tqdm(
            unit='B',
            unit_scale=True,
            total=file_size,
            desc=str(path)
    ) as progress:
        for chunk in r.iter_content(chunk_size=chunk_size):
            datasize = f.write(chunk)
            progress.update(datasize)


date = datetime.now().strftime("%d-%m-%Y")
release = '4.4.231'
base_path = Path('../data/multilayer_network/raw')
zip_path = base_path / f'BIOGRID-ORGANISM-{release}.tab3.zip'
specific_file = f'BIOGRID-ORGANISM-Homo_sapiens-{release}.tab3.txt'
txt_path = base_path / specific_file
output_json_path = Path(f'../data/multilayer_network/processed/BioGRID_interactions.{date}.json')

url = f'https://downloads.thebiogrid.org/Download/BioGRID/Release-Archive/BIOGRID-{release}/BIOGRID-ORGANISM-{release}.tab3.zip'

file_size = int(requests.head(url).headers.get('content-length', 0))

download_file(url, zip_path, file_size)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extract(specific_file, base_path)
os.remove(zip_path)

nodes = {}
edges = []

with open(txt_path, 'r') as file:
    next(file)  # Skip header
    for line in file:
        cols = line.strip().split('\t')
        if cols[15] == cols[16] == "9606" and cols[11] in protein_interaction_techniques:
            int1, int2 = cols[7], cols[8]
            nodes[int1] = {"label": int1, "type": "protein"}
            nodes[int2] = {"label": int2, "type": "protein"}
            edges.append({"source": int1, "target": int2, "attributes": {"Interaction Type": cols[12], "Experimental System": cols[11]}})

nodes_list = [{"id": k, **v} for k, v in nodes.items()]
final_structure = {"nodes": nodes_list, "edges": edges}

ensure_dir(output_json_path)

with open(output_json_path, 'w') as json_file:
    json.dump(final_structure, json_file, indent=4)

print(f"Interaction graph created and saved as JSON at {output_json_path}")

../data/multilayer_network/raw/BIOGRID-ORGANISM-4.4.231.tab3.zip: 169MB [00:11, 15.1MB/s] 


Interaction graph created and saved as JSON at ../data/multilayer_network/processed/BioGRID_interactions.23-09-2024.json


## 2. KEGG drugs

In [ ]:
def download_file_with_progress(url, filename):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        file_size = int(r.headers.get('content-length', 0))
        chunk_size = max(4096, file_size // 100)
        with open(filename, 'wb') as file_out, tqdm(
            total=file_size, unit='B', unit_scale=True, desc=str(filename)
        ) as pbar:
            for chunk in r.iter_content(chunk_size=chunk_size):
                file_out.write(chunk)
                pbar.update(len(chunk))

date = datetime.now().strftime("%d-%m-%Y")
release = 'br08310'
url = f'https://www.genome.jp/kegg-bin/download_htext?htext={release}.keg&format=json&filedir='

data_dir = Path('../data/multilayer_network/raw')
data_dir.mkdir(parents=True, exist_ok=True)
download_file_with_progress(url, data_dir / f'{release}.json')

def extract_drugs_and_receptors(node, drug_to_receptors):
    if 'children' in node:
        if 'name' in node and '[' in node['name']:
            receptor_name = re.sub(r'\s*\(.*?\)\s*', '', node['name'].split('[')[0].strip()).replace('*', '')
            for drug in node.get('children', []):
                drug_id, _, drug_name = drug['name'].partition(' ')
                drug_name = drug_name.lstrip()
                drug_to_receptors.setdefault(drug_id, {'receptors': [], 'drug_name': drug_name}).get('receptors').append(receptor_name)
        else:
            for child in node.get('children', []):
                extract_drugs_and_receptors(child, drug_to_receptors)

drug_to_receptors = {}
extract_drugs_and_receptors(json.loads((data_dir / f'{release}.json').read_text()), drug_to_receptors)

output = {
    "nodes": [
        {"id": receptor, "label": receptor, "type": "protein"} for receptor in {receptor for info in drug_to_receptors.values() for receptor in info['receptors']}
    ],
    "edges": [
        {"source": src, "target": tgt, "attributes": {"Drug ID": drug_id, "Drug Name": info['drug_name']}}
        for drug_id, info in drug_to_receptors.items() for src, tgt in combinations(set(info['receptors']), 2)
    ]
}

output_path = Path('../data/multilayer_network/processed') / f'KEGG_drugs.{date}.json'
output_path.parent.mkdir(parents=True, exist_ok=True)
output_path.write_text(json.dumps(output, indent=4))

print(f"JSON file created at {output_path}")

../data/multilayer_network/raw/br08310.json: 675kB [00:03, 207kB/s] 

JSON file created at ../data/multilayer_network/processed/KEGG_drugs.23-09-2024.json


## 3. MONDO diseases

URL address `http://purl.obolibrary.org/obo/mondo.owl` contains a 215 MB ontology, however the cell below doesn't create any processed dataset. Changed schema?

In [ ]:
def download_file(url, filename):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        total_size_in_bytes = int(r.headers.get('content-length', 0))
        progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True, desc="Downloading ontology")
        with open(filename, 'wb') as file:
            for chunk in r.iter_content(chunk_size=1024):
                progress_bar.update(file.write(chunk))
        progress_bar.close()

def extract_mondo2omim(g):
    rdfl = rdflib.Namespace('http://www.w3.org/2000/01/rdf-schema#')
    owl = rdflib.Namespace('http://www.w3.org/2002/07/owl#')
    oboInOwl = rdflib.Namespace('http://www.geneontology.org/formats/oboInOwl#')
    mondo2omim = {}
    mondo2descr = {}

    for subj in tqdm(g.subjects(), desc="Extracting subjects"):
        for i in g.triples((subj, owl['annotatedSource'], None)):
            mondo = i[2]
            mondoID = mondo.toPython().rsplit('/', 1)[-1]
            for j in g.triples((mondo, rdfl['label'], None)):
                descr = j[2].toPython()
                mondo2descr[mondoID] = descr
            for w in g.triples((mondo, oboInOwl['hasDbXref'], None)):
                if w[2].startswith(rdflib.Literal('OMIM:')):
                    omim = w[2].toPython()
                    mondo2omim[mondoID] = omim
    return mondo2omim, mondo2descr

def fetch_genes(url):
    try:
        df = pd.read_csv(url, sep='\t', header=0).dropna(subset=['evidence'])
        df = df[df['subject_taxon_label'].str.contains('Homo sapiens', na=False)]
        genes = df[df['evidence'].str.contains('ECO:0000220')].subject_label.unique().tolist()
        return genes
    except Exception as e:
        print(f"Failed to fetch genes from URL: {url} due to {e}")
        return []

def generate_json(mondo2genes, mondo2descr, output_file):
    nodes = [{"id": gene, "label": gene, "type": "gene"} for gene_list in mondo2genes.values() for gene in gene_list]
    edges = [{"source": gene_pair[0], "target": gene_pair[1], "attributes": {"MONDO Identifier": mondo, "Description": mondo2descr[mondo]}} 
             for mondo, genes in mondo2genes.items() for gene_pair in combinations(genes, 2)]

    final_json = {"nodes": list({v['id']:v for v in nodes}.values()), "edges": edges}
    with open(output_file, 'w') as json_file:
        json.dump(final_json, json_file, indent=4)

url = 'http://purl.obolibrary.org/obo/mondo.owl'
filename = '../data/multilayer_network/raw/mondo.owl'
output_path = Path("../data/multilayer_network/processed/MoNDO_diseases." + datetime.now().strftime("%d-%m-%Y") + ".json")


download_file(url, filename)
g = rdflib.Graph()
# g.load(filename)
g.parse(filename)
mondo2omim, mondo2descr = extract_mondo2omim(g)
    
inputs = [f"https://solr.monarchinitiative.org/solr/golr/select/?defType=edismax&qt=standard&indent=on&wt=csv&rows=100000&start=0&fl=subject,subject_label,subject_taxon,subject_taxon_label,object,object_label,relation,relation_label,evidence,evidence_label,source,is_defined_by,qualifier&facet=true&facet.mincount=1&facet.sort=count&json.nl=arrarr&facet.limit=25&facet.method=enum&csv.encapsulator=%22&csv.separator=%09&csv.header=true&csv.mv.separator=%7C&fq=subject_category:%22gene%22&fq=object_category:%22disease%22&fq=object_closure:%22{mondo.replace('_', ':')}%22&facet.field=subject_taxon_label&q=*:*" for mondo in mondo2omim.keys()]
with multiprocessing.Pool() as pool:
    genes_list = list(tqdm(pool.imap(fetch_genes, inputs), total=len(inputs), desc="Fetching genes"))
    
mondo2genes = {mondo: genes for mondo, genes in zip(mondo2omim.keys(), genes_list) if genes}
generate_json(mondo2genes, mondo2descr, output_path)
print(f"Generated JSON file at {output_path}")

## 4. Reactome pathways

In [ ]:
now = datetime.now()

# Download Uniprot id mapping
url = "https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/by_organism/HUMAN_9606_idmapping.dat.gz"
response = requests.get(url)
filename = 'data/multilayer_network/raw/HUMAN_9606_idmapping.dat.gz'
outname = 'data/multilayer_network/raw/HUMAN_9606_idmapping.dat'
with open(filename, 'wb') as file:
    file.write(response.content)
with gzip.open(filename, 'rb') as f_in:
    with open(outname, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
os.remove(filename)
mapping_dict = {}
with open(outname, 'r') as file:
    for line in file:
        parts = line.strip().split()
        uni_id, identifier_type, value = parts[0], parts[1], ' '.join(parts[2:])
        if identifier_type == 'Gene_Name':
            mapping_dict[uni_id] = value

# Download Reactome lowest levels
url = 'https://reactome.org/download/current/UniProt2Reactome.txt'
response = requests.get(url)
filename = 'data/multilayer_network/raw/UniProt2Reactome.txt'
with open(filename, 'wb') as file:
    file.write(response.content)

nodes = []
edges = []

reactome = {}
descr_dict = {}
with open(filename, 'r') as file:
    for line in file:
        line = line.strip()
        parts = line.split('\t')
        if len(parts) < 6:
            continue  # Skip incomplete lines
        prot, pathway, descr, ec, taxo = parts[0], parts[1], parts[3], parts[4], parts[5]
        descr_dict[pathway] = descr
        if taxo != "Homo sapiens":
            continue
        # Skip certain evidence codes if needed
        # if ec not in ["EXP", "IDA", "IPI", "IMP", "IGI", "IEP"]:
        #    continue
        if pathway not in reactome:
            reactome[pathway] = []
        if prot not in reactome[pathway]:
            try:
                gene = mapping_dict[prot]
            except:
                gene = prot
            reactome[pathway].append(gene)
        if gene not in [node['id'] for node in nodes]:
            nodes.append({
                "id": gene,
                "label": gene,
                "type": "protein"
            })

for k,v in reactome.items():
    if len(v) > 1:
        for gene_pair in combinations(v, 2):
            edges.append({
                "source": gene_pair[0],
                "target": gene_pair[1],
                "attributes": {
                    "Reactome Identifier": k,
                    "Description": descr_dict[k]
                }
            })

final_json = {"nodes": nodes, "edges": edges}
file_name = "data/multilayer_network/processed/Reactome_pathways." + now.strftime("%d-%m-%Y") + ".json"
with open(file_name, 'w') as json_file:
    json.dump(final_json, json_file, indent=4)

## 5. Recon3D metabolites

In [ ]:
# Use the current timestamp for naming the output file later
now = datetime.now().strftime("%d-%m-%Y")

# Function to download and decompress the model file
def download_and_decompress(url, output_path):
    with urllib.request.urlopen(url) as response, open(output_path, 'wb') as out_file:
        out_file.write(gzip.decompress(response.read()))

# Define URLs and file paths
model_url = 'http://bigg.ucsd.edu/static/models/Recon3D.xml.gz'
model_file_path = 'data/multilayer_network/raw/Recon3D.xml'
metabolites_file_path = 'src/metabolites_to_prune.txt'

# Download and import the Recon3D model
download_and_decompress(model_url, model_file_path)
model = cobra.io.read_sbml_model(model_file_path)

# Import the metabolites to prune
with open(metabolites_file_path) as f:
    remove_metabolites = [line.split(' ')[0].strip() for line in f]

# Function to process metabolites and genes
def process_metabolites_and_genes(model, remove_metabolites):
    products_dict, reactants_dict, metabo_name_dict = defaultdict(list), defaultdict(list), {}
    gene_pattern, metabo_pattern = r'__[clmerginx]$', r'__\d+__\d+$'

    for reaction in model.reactions:
        for gene in filter(lambda g: g.id.split('_')[0] != "0", reaction.genes):
            gene_id = gene.id.split('_')[0]
            for metabolite_group, metabolites in (('products', reaction.products), ('reactants', reaction.reactants)):
                for metabolite in metabolites:
                    metabo_id = re.sub(gene_pattern, '', metabolite.id)
                    metabo_name_dict[metabo_id] = metabolite.name
                    if gene_id not in (products_dict if metabolite_group == 'products' else reactants_dict)[metabo_id]:
                        (products_dict if metabolite_group == 'products' else reactants_dict)[metabo_id].append(gene.name)

    metabolites_dict = defaultdict(list)
    for metabo_id, genes in {**products_dict, **reactants_dict}.items():
        if metabo_id not in remove_metabolites:
            metabolites_dict[re.sub(metabo_pattern, '', metabo_id)].extend(set(genes))

    return {k: list(set(v)) for k, v in metabolites_dict.items()}, metabo_name_dict

metabolites_dict, metabo_name_dict = process_metabolites_and_genes(model, remove_metabolites)

# Preparing data for JSON output
nodes = [{"id": gene, "label": gene, "type": "protein"} for gene in set(gene for genes in metabolites_dict.values() for gene in genes)]
edges = [{
    "source": gene_pair[0],
    "target": gene_pair[1],
    "attributes": {"Metabolite": metabo, "Descriptive name": metabo_name_dict[metabo]}
} for metabo, genes in metabolites_dict.items() for gene_pair in combinations(genes, 2)]

# Saving the graph to a JSON file
output_file_path = f'data/multilayer_network/processed/Recon3D_metabolites.{now}.json'
with open(output_file_path, 'w') as f:
    json.dump({"nodes": nodes, "edges": edges}, f, indent=4)

print(f"Graph saved to {output_file_path}")
